In [1]:
import warnings
warnings.filterwarnings('ignore')
import picaso.justdoit as jdi
import astropy.units as u

import numpy as np
import time
import matplotlib.pyplot as plt

In [2]:
mh = '+0.0'
CtoO = '1.0'


filename_db="/Users/sagnickmukherjee/Documents/GitHub/picaso-dev/reference/opacities/ck_cx_cont_opacities.db"
ck_db='/Users/sagnickmukherjee/Documents/software/picaso-dev/reference/opacities/ck_db/sonora_2020_feh+000_co_100.data.196'

opacity_ck = jdi.opannection(ck=True, ck_db=ck_db,filename_db=filename_db)


In [3]:
#[500,700, 800, 900, 1000,1300], g = [100, 178, 316, 562, 1000], and log Kzz = [3,4, 5, 6, 7, 8,9]. 


In [4]:
Teff_ar = np.array([600])
grav_ar = np.array([1000])
kzz=np.array([1e3,1e4,1e5,1e6,1e7,1e8,1e9,1e10])

for iii in range(len(kzz)):
    T_eff= Teff_ar[0]
    grav = grav_ar[0]
    nlevel = 91

    ## Brown Dwarf 
    T_star = "NA"
    semi_major = "NA" # AU
    self_consistent_kzz= False # Only needed if you want to do on-the-fly calculation with MLT Kzz
    cl_run = jdi.inputs(calculation="brown", climate = True)

    cl_run.gravity(gravity=grav, gravity_unit=u.Unit('m/(s**2)'))
    cl_run.T_eff(Teff=T_eff)




    #pressure=np.logspace(np.log10(1.77000392e-04),np.log10(80.0),nlevel) # set your pressure grid
    temp_guess = np.zeros(shape=(nlevel))
    kz = np.zeros(shape =(nlevel))

    ## load a bobcat model
    pressure_dummy,temp_guess = np.loadtxt("/Users/sagnickmukherjee/Documents/software/picaso-dev/Evolution/structures_m+0.0/t"+str(Teff_ar[0])+"g"+str(grav_ar[0])+"nc_m0.0.dat",usecols=[1,2],unpack=True, skiprows = 1)
    #pressure=pressure_dummy
    pressure = np.logspace(np.log10(1.7e-4),np.log10(1000),91)
    for i in range(len(temp_guess)):
        temp_guess[i] = temp_guess[i] # perturb the profile
        kz[i] = kzz[iii] #0

    nofczns = 1
    nstr = np.array([0,74,89,0,0,0]) # initial guess of convective zones
    rfacv = 0.0
    rfaci =1.0
    cloudy = False # not cloudy
    fsed = 0.0
        ##
    cl_run.inputs_climate(temp_guess= temp_guess, pressure= pressure, nstr = nstr, nofczns = nofczns , rfacv = rfacv, rfaci = rfaci, cloudy = cloudy, mh = mh, CtoO = CtoO,species = ['MgSiO3','Mg2SiO4','Fe'], fsed = fsed)
    pressure , temp, dtdp, nstr_new, flux_plus_final,quench_levels, df, all_profiles, all_kzz,opd_,g0_,w0_ = cl_run.run_climate_model(opacity_ck,save_all_profiles=True, save_all_kzz = True,diseq_chem = True, self_consistent_kzz =self_consistent_kzz, kz = kz,vulcan_run = False)
    
    
    path = "/Users/sagnickmukherjee/Documents/research/geers/51erib/"

    filename= path+"tpfile/tp_deq_planet_"+str(T_eff)+"_grav_"+str(grav)+"_1e"+str(int(np.log10(kzz[iii])))+".dat"
    np.savetxt(filename, np.transpose([pressure,temp]), header= str("nstr "+str(nstr_new)+" T_eff "+str(T_eff)+" grav "+ str(grav)+" kz "+str(kz[0])+" T_star "+str(T_star)+" semi-major[AU] "+str(semi_major)))        

    csv = path+"profile/profile_deq_planet_"+str(T_eff)+"_grav_"+str(grav)+"_1e"+str(int(np.log10(kzz[iii])))+".dat"
    df.to_csv(csv,sep='\t')

    hist = path+"hist/hist_deq_planet_"+str(T_eff)+"_grav_"+str(grav)+"_1e"+str(int(np.log10(kzz[iii])))+".dat"
    np.savetxt(hist, all_profiles)

    hist = path+"hist/hist_kz_deq_planet_"+str(T_eff)+"_grav_"+str(grav)+"_1e"+str(int(np.log10(kzz[iii])))+".dat"
    np.savetxt(hist, all_kzz)

Iteration number  0 , min , max temp  172.9765462502491 2626.287040649518 , flux balance  -0.5082766711432339
Iteration number  1 , min , max temp  169.8017562052438 2892.967465250661 , flux balance  -0.18907541222666016
Iteration number  2 , min , max temp  167.249267250243 2982.922602529617 , flux balance  0.020079473957700933
Iteration number  3 , min , max temp  167.4124112891074 2974.770917899354 , flux balance  0.00019235185959380334
Converged Solution in iterations  3
Big iteration is  167.4124112891074 0
Iteration number  0 , min , max temp  179.67884886869467 3018.896826807547 , flux balance  0.00752255898015313
Iteration number  1 , min , max temp  178.39231458536716 3018.007901903098 , flux balance  3.3517122643644654e-05
Iteration number  2 , min , max temp  178.36600786791243 3018.0065988922584 , flux balance  1.0162487058246581e-08
Converged Solution in iterations  2
Big iteration is  178.36600786791243 1
Iteration number  0 , min , max temp  171.10581033283302 3022.86045

Profile converged
[ 0 62 67 67 78 89]
Iteration number  0 , min , max temp  130.91688134610513 2945.220466647605 , flux balance  3.4741604800946736e-05
Converged Solution in iterations  0
Quench Levels are CO, CO2, NH3, HCN  [69 63 72 68]
Big iteration is  130.91688134610513 0
Iteration number  0 , min , max temp  130.64279270218321 2945.037382589637 , flux balance  2.623086676272289e-06
Converged Solution in iterations  0
Quench Levels are CO, CO2, NH3, HCN  [69 63 72 68]
Profile converged
[ 0 61 67 67 78 89]
Iteration number  0 , min , max temp  130.67204472758132 2944.64458266908 , flux balance  7.524959766384747e-05
Iteration number  1 , min , max temp  130.67410708999572 2944.6414092116534 , flux balance  1.8610648367943376e-07
Converged Solution in iterations  1
Quench Levels are CO, CO2, NH3, HCN  [69 63 72 68]
Big iteration is  130.67410708999572 0
Iteration number  0 , min , max temp  130.6664095417342 2944.621820640946 , flux balance  -4.45702847058186e-06
Converged Solution 

Big iteration is  167.4124112891074 0
Iteration number  0 , min , max temp  179.67884886869467 3018.896826807547 , flux balance  0.00752255898015313
Iteration number  1 , min , max temp  178.39231458536716 3018.007901903098 , flux balance  3.3517122643644654e-05
Iteration number  2 , min , max temp  178.36600786791243 3018.0065988922584 , flux balance  1.0162487058246581e-08
Converged Solution in iterations  2
Big iteration is  178.36600786791243 1
Iteration number  0 , min , max temp  171.10581033283302 3022.860455192561 , flux balance  0.000339599846541218
Iteration number  1 , min , max temp  170.34383105386217 3022.839926926068 , flux balance  1.3736883590239259e-07
Converged Solution in iterations  1
Profile converged
Iteration number  0 , min , max temp  176.17872882652514 3023.5380286180794 , flux balance  1.2399837812788687e-05
 We are already at a root, tolf , test =  5e-05 ,  1.4395418323371994e-05
Big iteration is  176.17872882652514 0
Iteration number  0 , min , max temp  1

Profile converged
[ 0 61 67 67 78 89]
Iteration number  0 , min , max temp  133.19248830918627 2945.4629645042187 , flux balance  2.2905726090878522e-05
Converged Solution in iterations  0
Quench Levels are CO, CO2, NH3, HCN  [70 64 73 69]
Big iteration is  133.19248830918627 0
Iteration number  0 , min , max temp  133.26514839328007 2945.3629251643647 , flux balance  -2.403427020849236e-06
Converged Solution in iterations  0
Quench Levels are CO, CO2, NH3, HCN  [70 64 73 69]
Profile converged
[ 0 60 67 67 78 89]
Iteration number  0 , min , max temp  133.40807816483584 2945.103162966884 , flux balance  3.2678267181087673e-05
Iteration number  1 , min , max temp  133.4107174336954 2945.0997112917844 , flux balance  -3.082973027882949e-09
Converged Solution in iterations  1
Quench Levels are CO, CO2, NH3, HCN  [70 64 73 69]
Big iteration is  133.4107174336954 0
Iteration number  0 , min , max temp  133.48959626073054 2945.1038406816547 , flux balance  -6.330198902091468e-06
Converged Sol

Big iteration is  178.36600786791243 1
Iteration number  0 , min , max temp  171.10581033283302 3022.860455192561 , flux balance  0.000339599846541218
Iteration number  1 , min , max temp  170.34383105386217 3022.839926926068 , flux balance  1.3736883590239259e-07
Converged Solution in iterations  1
Profile converged
Iteration number  0 , min , max temp  176.17872882652514 3023.5380286180794 , flux balance  1.2399837812788687e-05
 We are already at a root, tolf , test =  5e-05 ,  1.4395418323371994e-05
Big iteration is  176.17872882652514 0
Iteration number  0 , min , max temp  173.28228039121018 3023.658277537909 , flux balance  3.7056506102347355e-07
 We are already at a root, tolf , test =  5e-05 ,  4.917480367670639e-07
Profile converged
 convection zone status
0 74 89 0 0 0
1
[ 0 64 64 64 74 89]
Iteration number  0 , min , max temp  173.95267245971178 3023.672139247551 , flux balance  4.678243745948788e-07
 We are already at a root, tolf , test =  5e-05 ,  1.235651217162069e-05
Bi

...

Profile converged
[ 0 53 67 67 78 89]
Iteration number  0 , min , max temp  148.62848647904582 2941.775851408616 , flux balance  -2.5950543649033903e-05
Converged Solution in iterations  0
Quench Levels are CO, CO2, NH3, HCN  [77 69 79 77]
Big iteration is  148.62848647904582 0
Iteration number  0 , min , max temp  148.67533226813688 2941.793050942231 , flux balance  -7.074908195282525e-06
Converged Solution in iterations  0
Quench Levels are CO, CO2, NH3, HCN  [77 69 79 77]
Profile converged
[ 0 53 68 68 78 89]
Iteration number  0 , min , max temp  148.6732699930666 2941.8981438100514 , flux balance  3.1807426369348514e-05
Converged Solution in iterations  0
Quench Levels are CO, CO2, NH3, HCN  [77 69 79 77]
Big iteration is  148.6732699930666 0
Iteration number  0 , min , max temp  148.67090009312815 2941.890379744142 , flux balance  1.5572638491398641e-06
Converged Solution in iterations  0
Quench Levels are CO, CO2, NH3, HCN  [77 69 79 77]
Profile converged
[ 0 53 68 68 77 89]
Iter

Big iteration is  151.09255041390153 0
Iteration number  0 , min , max temp  151.1246294030725 2942.863228178121 , flux balance  1.5261686787473928e-07
Converged Solution in iterations  0
Quench Levels are CO, CO2, NH3, HCN  [79 70 80 79]
Profile converged
Grow Phase : Upper Zone
[ 0 63 64 64 78 89]
Iteration number  0 , min , max temp  151.16098916393364 2942.804622257718 , flux balance  1.2749047934197552e-05
Converged Solution in iterations  0
Quench Levels are CO, CO2, NH3, HCN  [79 70 80 79]
Big iteration is  151.16098916393364 0
Iteration number  0 , min , max temp  151.1684942527598 2942.785783073652 , flux balance  7.282769473579674e-08
Converged Solution in iterations  0
Quench Levels are CO, CO2, NH3, HCN  [79 70 80 79]
Profile converged
[ 0 62 64 64 78 89]
Iteration number  0 , min , max temp  151.23250693006577 2942.7043648800477 , flux balance  2.0440878612756745e-05
Iteration number  1 , min , max temp  151.23256837510633 2942.7043569822454 , flux balance  1.2579388892081

Big iteration is  153.2040678404111 0
Iteration number  0 , min , max temp  153.17105182938352 2945.6518486356526 , flux balance  1.674183110206884e-06
Converged Solution in iterations  0
Quench Levels are CO, CO2, NH3, HCN  [79 70 80 79]
Profile converged
[ 0 52 67 67 76 89]
Iteration number  0 , min , max temp  153.29488815318996 2955.0217749862554 , flux balance  8.183101297951065e-05
Converged Solution in iterations  0
Quench Levels are CO, CO2, NH3, HCN  [79 70 80 79]
Big iteration is  153.29488815318996 0
Iteration number  0 , min , max temp  153.20556995219414 2955.1158386693132 , flux balance  8.832414453854981e-07
Converged Solution in iterations  0
Quench Levels are CO, CO2, NH3, HCN  [79 70 80 79]
Profile converged
[ 0 52 67 67 75 89]
Iteration number  0 , min , max temp  153.37527206279267 2967.647791017366 , flux balance  0.0001314835905029063
Converged Solution in iterations  0
Quench Levels are CO, CO2, NH3, HCN  [79 70 80 78]
Big iteration is  153.37527206279267 0
Itera

Big iteration is  153.62514065466803 0
Iteration number  0 , min , max temp  153.6458389040583 2944.7453869820424 , flux balance  -2.368009154159017e-08
Converged Solution in iterations  0
Quench Levels are CO, CO2, NH3, HCN  [81 72 82 81]
Profile converged
[ 0 61 64 64 78 89]
Iteration number  0 , min , max temp  153.7013993606649 2944.687466393687 , flux balance  1.2354173003017645e-05
Converged Solution in iterations  0
Quench Levels are CO, CO2, NH3, HCN  [81 72 82 81]
Big iteration is  153.7013993606649 0
Iteration number  0 , min , max temp  153.72999407024824 2944.686678531132 , flux balance  -2.9761148989649086e-07
Converged Solution in iterations  0
Quench Levels are CO, CO2, NH3, HCN  [81 72 82 81]
Profile converged
[ 0 61 65 65 78 89]
Iteration number  0 , min , max temp  153.81707182120644 2944.594422387756 , flux balance  3.941828090535963e-06
Converged Solution in iterations  0
Quench Levels are CO, CO2, NH3, HCN  [81 72 82 81]
Big iteration is  153.81707182120644 0
Itera

Big iteration is  155.5379487906464 0
Iteration number  0 , min , max temp  155.54118923304503 2956.9201524806945 , flux balance  -4.414575762097721e-08
Converged Solution in iterations  0
Quench Levels are CO, CO2, NH3, HCN  [81 72 82 81]
Profile converged
YAY ! ENDING WITH CONVERGENCE
